In [1]:
from web3 import Web3
from hexbytes import HexBytes
import pandas as pd
import numpy as np
import rlp
import requests
from web3 import Web3
from dotenv import load_dotenv
from base_bot_parser import BaseBotParser

from datetime import datetime, timedelta, timezone

import matplotlib.pylab as plt
import numpy as np
import json
import logging
import typing
import requests
import sys
import os
import time
import pickle

import pandas as pd
import seaborn as sns
from dotenv import load_dotenv
from web3 import Web3

from EtherScan import EtherScan
from FortaExplorer import FortaExplorer
from Plotter import Plotter

load_dotenv()


root = logging.getLogger()
root.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

rpc_endpoint_address=os.environ.get('MAINNET_INFURA_RPC_ENDPOINT')
w3 = Web3(Web3.HTTPProvider(rpc_endpoint_address))
w3.manager.request_blocking


<bound method RequestManager.request_blocking of <web3.manager.RequestManager object at 0x7fb7916a61c0>>

In [2]:
df_etherscan_addresses = pd.read_csv(f"end_user_attacks/20230517_addresses_allium.csv")
df_etherscan_addresses = df_etherscan_addresses[df_etherscan_addresses['is_contract'] == False]
df_etherscan_addresses.rename(columns={'attacker_address': 'address'}, inplace=True)
print(len(df_etherscan_addresses))
df_benign_addresses = pd.read_csv(f"end_user_attacks/20230517_benign_addresses_selected.csv")
df_fps_addresses = pd.read_csv(f"end_user_attacks/20230517_fps.csv")
df_benign_addresses = pd.concat([df_benign_addresses, df_fps_addresses])
print(len(df_benign_addresses))

2188
1830


In [3]:
def convert_metadata_into_dict(metadata_str: str):
    dictionary = json.loads(metadata_str)

    metadata = {}
    key = ''
    value = ''
    for item in dictionary:
        for sub_item in item['map']:
            if sub_item['key'] == 'value':
                value = sub_item['value']
            if sub_item['key'] == 'key':
                key = sub_item['value']
                metadata[key] = value
            
    return metadata


In [4]:
def is_attacker_alert(row: pd.Series) -> bool:
    address = row['attacker_address']
    bot_id = row['bot_id']
    alert_id = row['alert_id']
    description = row['description']
    metadata = row['metadata']

    scammer_addresses = BaseBotParser.get_scammer_addresses(w3, bot_id, alert_id, description, metadata)

    if address in scammer_addresses:
        return True
    else:
        return False
    

In [5]:
def concat_dfs(df_addresses: pd.DataFrame, name: str) -> pd.DataFrame:
    df_alerts_all = pd.DataFrame(columns=['attacker_address', 'first_tx_date', 'alert_date', 'alert_hash', 'bot_id', 'alert_id',
        'description', 'metadata', 'is_attacker_alert'])
    

    for index, row in df_addresses.iterrows():
        #if file exists
        if os.path.isfile(f"end_user_attacks/{row['address']}_HOVPiHkG5qVy0EwFDkjo.pkl"):
            df_alerts = pickle.load(open(f"end_user_attacks/{row['address']}_HOVPiHkG5qVy0EwFDkjo.pkl", "rb"))


            # removing alerts that are tied to labels or just too noisy
            if len(df_alerts) > 0:
                df_alerts = df_alerts[~df_alerts['alert_id'].str.contains('GNOSIS')]
                df_alerts = df_alerts[~df_alerts['alert_id'].str.contains('SCAM')]
                print (f"{name}: Loaded {row['address']} alerts: {len(df_alerts)}")
            else:
                print (f"{name}: Loaded {row['address']} alerts emtpy")


            # the alert data is a broad selection based on:
            # (ARRAY_CONTAINS('{{address_lower}}'::variant, addresses)
            # or ARRAY_TO_STRING(metadata,',') ilike '%{{address_lower}}%'
            # or description ilike '%{{address_lower}}%')
            # need to narrow it down based on the bot parsing logic in scam detector

            print(df_alerts.shape)

            if len(df_alerts) > 0:
                df_alerts['is_attacker_alert'] = df_alerts.apply(lambda x: is_attacker_alert(x), axis=1)
                df_alerts = df_alerts[df_alerts['is_attacker_alert'] == True]

                df_alerts_all = pd.concat([df_alerts_all, df_alerts], ignore_index=True)
    df_alerts_all.rename(columns={'attacker_address': 'address'}, inplace=True)
    return df_alerts_all

df_alerts_malicious = concat_dfs(df_etherscan_addresses, "Malicious")
df_alerts_benign = concat_dfs(df_benign_addresses, "Benign")

df_alerts_malicious['label'] = 1
df_alerts_benign['label'] = 0
df_alerts_all = pd.concat([df_alerts_malicious, df_alerts_benign], ignore_index=True)

Malicious: Loaded 0xc16acf1380a144f673411226c9dd913cf1eec182 alerts: 1866
(1866, 8)
Malicious: Loaded 0xb3744fcf41d71d5178f391792abe63fe5d638b7e alerts: 0
(0, 8)
Malicious: Loaded 0xecba0d1eee95a6ea8831d1179e16046749509e79 alerts: 0
(0, 8)
Malicious: Loaded 0x6bead068a21acec03c0f5e81a0a4a9636420bcfc alerts: 276
(276, 8)
Malicious: Loaded 0x124ba0671024cc8d3df23cee5da034076797b7a2 alerts: 0
(0, 8)
Malicious: Loaded 0x8009194614a7b3bca888fc8f994d8545050b9d5e alerts: 11
(11, 8)
Malicious: Loaded 0xaac4811a8bedd72b88a6aff235c5d0e9318ef935 alerts: 0
(0, 8)
Malicious: Loaded 0x85bc0d58556f41f0463927c9dcb81d190f452610 alerts: 24
(24, 8)
Malicious: Loaded 0x8413c544095f7e69d4ad3bfcf4b77deedda0cc39 alerts: 0
(0, 8)
Malicious: Loaded 0x669701fdaf54058e6498aba4004221d400ba5ff8 alerts: 186
(186, 8)
Malicious: Loaded 0x86898c3b7685f789f82853a1c3b25ed5c5dffbd3 alerts: 96
(96, 8)
Malicious: Loaded 0x1b37e31fd7e6438f69b505ec7dd93f4a5bb77fc8 alerts: 4
(4, 8)
Malicious: Loaded 0x39bc02af76ca261b9bc1ff9b

In [6]:
def summarize(df_alerts_all: pd.DataFrame) -> pd.DataFrame:
    # filter for the alerts raised in the first 30 days of the attacker address first tx
    df_alerts_all['first_tx_date'] = pd.to_datetime(df_alerts_all['first_tx_date'])
    df_alerts_all['alert_date'] = pd.to_datetime(df_alerts_all['alert_date'])
    df_alerts_all['within_date_range'] = df_alerts_all.apply(lambda row: (row['alert_date'] - row['first_tx_date']).days < 30, axis=1)
    df_alerts_all = df_alerts_all[df_alerts_all['within_date_range'] == True]

    #remove alerts that do not contain the attacker address and drop remaining columns
    df_alerts_all.drop(columns=['is_attacker_alert', 'description', 'metadata'], inplace=True)

    #unique it based on alert_hash
    df_alerts_all.drop_duplicates(inplace=True)

    #aggregate alerts by bot_id and alert_id
    df_alerts_all = df_alerts_all.groupby(['label', 'address', 'bot_id', 'alert_id'])['alert_hash'].count().reset_index()
    return df_alerts_all

df_alerts_all_summary = summarize(df_alerts_all)
df_alerts_all_summary_without_label = df_alerts_all_summary.drop(columns=['label'])


/var/folders/cs/cnhc1qkj7v581034p3cly0nw0000gn/T/ipykernel_35946/4009790794.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_alerts_all.drop(columns=['is_attacker_alert', 'description', 'metadata'], inplace=True)
/var/folders/cs/cnhc1qkj7v581034p3cly0nw0000gn/T/ipykernel_35946/4009790794.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_alerts_all.drop_duplicates(inplace=True)


In [7]:
#df_alerts_all_summary.set_index(['address'], inplace=True)
df_labels = df_alerts_all_summary.drop(columns=['bot_id','alert_id','alert_hash'])
df_labels.drop_duplicates(inplace=True)
df_labels.set_index(['address'], inplace=True)
df_labels

,label
address,
0x0000000625bda2f61151f7ca5aea0b5ddacebaf0,0
0x000000697bb288f2528042e8844b65cd32beafca,0
0x0005f82d812aa31da5185565a8d691ab15142678,0
0x00270413ad6b01855e4e2186e5f32f51617bae49,0
0x005d6f11124eb923419bb47b52752a0cb44d424e,0
...,...
0xfb8cdb89c69779b608aae967538cd7b16cd13e3d,1
0xfd6ade3c2a97f2c8c4581b152197f6efaec41c25,1
0xfe7f1b2d6d391b4dacea559a5cd99dd0e8b90c5e,1


In [8]:
# Group by id, event and subevent and sum the count column
grouped = df_alerts_all_summary_without_label.groupby(['address', 'bot_id', 'alert_id'])['alert_hash'].sum().reset_index()

# Pivot the table to create columns for each event/subevent combination
pivoted = pd.pivot_table(grouped, values='alert_id', index='address', columns=['bot_id', 'alert_id'], aggfunc='sum')

# Rename the columns to use event_subevent format
pivoted.columns = [f'{col[0]}_{col[1]}' for col in pivoted.columns]

# Fill any missing values with 0
pivoted.fillna(0, inplace=True)

# join to get label back
df_training = pivoted.join(df_labels)
df_training.drop_duplicates(inplace=True)

df_training.columns

Index(['0x067e4c4f771f288c686efa574b685b98a92918f038a478b82c9ac5b5b6472732_NFT-WASH-TRADE',
       '0x186f424224eac9f0dc178e32d1af7be39506333783eec9463edd247dc8df8058_FLD_FUNDING',
       '0x186f424224eac9f0dc178e32d1af7be39506333783eec9463edd247dc8df8058_FLD_Laundering',
       '0x186f424224eac9f0dc178e32d1af7be39506333783eec9463edd247dc8df8058_FLD_NEW_FUNDING',
       '0x1a69f5ec8ef436e4093f9ec4ce1a55252b7a9a2d2c386e3f950b79d164bc99e0_NIP-1',
       '0x1a69f5ec8ef436e4093f9ec4ce1a55252b7a9a2d2c386e3f950b79d164bc99e0_NIP-2',
       '0x1a69f5ec8ef436e4093f9ec4ce1a55252b7a9a2d2c386e3f950b79d164bc99e0_NIP-4',
       '0x1a69f5ec8ef436e4093f9ec4ce1a55252b7a9a2d2c386e3f950b79d164bc99e0_NIP-6',
       '0x2e51c6a89c2dccc16a813bb0c3bf3bbfe94414b6a0ea3fc650ad2a59e148f3c8_ANOMALOUS-TOKEN-TRANSFERS-TX',
       '0x2e51c6a89c2dccc16a813bb0c3bf3bbfe94414b6a0ea3fc650ad2a59e148f3c8_INVALID-TOKEN-TRANSFERS-TX',
       '0x2e51c6a89c2dccc16a813bb0c3bf3bbfe94414b6a0ea3fc650ad2a59e148f3c8_NORMAL-TOKEN-TRAN

In [9]:
df_training

,0x067e4c4f771f288c686efa574b685b98a92918f038a478b82c9ac5b5b6472732_NFT-WASH-TRADE,0x186f424224eac9f0dc178e32d1af7be39506333783eec9463edd247dc8df8058_FLD_FUNDING,0x186f424224eac9f0dc178e32d1af7be39506333783eec9463edd247dc8df8058_FLD_Laundering,0x186f424224eac9f0dc178e32d1af7be39506333783eec9463edd247dc8df8058_FLD_NEW_FUNDING,0x1a69f5ec8ef436e4093f9ec4ce1a55252b7a9a2d2c386e3f950b79d164bc99e0_NIP-1,0x1a69f5ec8ef436e4093f9ec4ce1a55252b7a9a2d2c386e3f950b79d164bc99e0_NIP-2,0x1a69f5ec8ef436e4093f9ec4ce1a55252b7a9a2d2c386e3f950b79d164bc99e0_NIP-4,0x1a69f5ec8ef436e4093f9ec4ce1a55252b7a9a2d2c386e3f950b79d164bc99e0_NIP-6,0x2e51c6a89c2dccc16a813bb0c3bf3bbfe94414b6a0ea3fc650ad2a59e148f3c8_ANOMALOUS-TOKEN-TRANSFERS-TX,0x2e51c6a89c2dccc16a813bb0c3bf3bbfe94414b6a0ea3fc650ad2a59e148f3c8_INVALID-TOKEN-TRANSFERS-TX,...,0xe4a8660b5d79c0c64ac6bfd3b9871b77c98eaaa464aa555c00635e9d8b33f77f_ASSET-DRAINED,0xf234f56095ba6c4c4782045f6d8e95d22da360bdc41b75c0549e2713a93231a4_SOFT-RUG-PULL-SUS-LIQ-POOL-CREATION,0xf234f56095ba6c4c4782045f6d8e95d22da360bdc41b75c0549e2713a93231a4_SOFT-RUG-PULL-SUS-LIQ-POOL-CREATION && SOFT-RUG-PULL-SUS-LIQ-POOL-RESERVE-CHANGE,0xf234f56095ba6c4c4782045f6d8e95d22da360bdc41b75c0549e2713a93231a4_SOFT-RUG-PULL-SUS-LIQ-POOL-RESERVE-CHANGE,0xf234f56095ba6c4c4782045f6d8e95d22da360bdc41b75c0549e2713a93231a4_SOFT-RUG-PULL-SUS-LIQ-POOL-RESERVE-CHANGE && SOFT-RUG-PULL-SUS-LIQ-POOL-CREATION,0xf234f56095ba6c4c4782045f6d8e95d22da360bdc41b75c0549e2713a93231a4_SOFT-RUG-PULL-SUS-LIQ-POOL-RESERVE-CHANGE && SOFT-RUG-PULL-SUS-POOL-REMOVAL,0xf234f56095ba6c4c4782045f6d8e95d22da360bdc41b75c0549e2713a93231a4_SOFT-RUG-PULL-SUS-POOL-REMOVAL,0xf234f56095ba6c4c4782045f6d8e95d22da360bdc41b75c0549e2713a93231a4_SOFT-RUG-PULL-SUS-POOL-REMOVAL && SOFT-RUG-PULL-SUS-LIQ-POOL-RESERVE-CHANGE,0xf496e3f522ec18ed9be97b815d94ef6a92215fc8e9a1a16338aee9603a5035fb_CEX-FUNDING-1,label
address,,,,,,,,,,,,,,,,,,,,,
0x00000000fffb6508e8c3db931fefc3895cf36d15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
0x00000001c64889f0ad579993ecd65e97863608c1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
0x0000000625bda2f61151f7ca5aea0b5ddacebaf0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
0x000000082b50ddb583f57c338049f2acbda2712a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
0x0000000d5377bff1cb2e310362d0bbec826fb425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0xfd4ba3881d78241060bd196b5819c8e618d22d51,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
0xfd6ade3c2a97f2c8c4581b152197f6efaec41c25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
0xfe7f1b2d6d391b4dacea559a5cd99dd0e8b90c5e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [10]:
len(df_training)

676

In [11]:
df_training.to_csv("end_user_attacks/20230517_fp_df_training.csv")
pickle.dump(df_training, open("end_user_attacks/20230517_fp_df_training.pkl", "wb"))

In [12]:
#TODO - normalize by number of transactions in the first 30 days
#TODO - extract score from ML models and output the max/min of the scores